**REGRESIÓN PARTIENDO DE IMÁGENES**

En este notebook vamos a tratar de hacer una regresión para estimar el precio de un apartamento de alquiler. 
Dicha predicción se va a hacer a partir de las imágenes que nos descargamos del dataset de airbnb que venimos usando en las prácticas de este Bootcamp.

En primer lugar nos descargamos el fichero de internet y lo copiamos en un directorio local de My Drive donde tenemos recogido todo el entorno de esta práctica.
A continuación hacemos lo mismo con las imágenes de cada una de las entradas. Usamos la vista en miniatura que sacamos de la URL de dicho fichero.

También montamos el google collab con My Drive para tenerlo vinculado.

Estos pasos solo hay que realizarlos la primera vez, una vez que tenemos los ficheros en My Drive se pueden saltar y pasamos a cargar los datos directamente desde dicho directorio.

In [ ]:
# nos descargamos el dataset de OpenDataSoft
!wget -O "airbnb-listings.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"

!ls -lah

--2020-06-25 06:01:03--  https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B
Resolving public.opendatasoft.com (public.opendatasoft.com)... 34.249.199.226, 34.248.20.69
Connecting to public.opendatasoft.com (public.opendatasoft.com)|34.249.199.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘airbnb-listings.csv’

airbnb-listings.csv     [  <=>               ]  54.19M  2.77MB/s    in 50s     

2020-06-25 06:02:01 (1.09 MB/s) - ‘airbnb-listings.csv’ saved [56826824]

total 55M
drwxr-xr-x 1 root root 4.0K Jun 25 06:01 .
drwxr-xr-x 1 root root 4.0K Jun 25 05:59 ..
-rw-r--r-- 1 root root  55M Jun 25 06:02 airbnb-listings.csv
drwxr-xr-x 1 root root 4.0K Jun 19 16:15 .config
drwx------ 4 root root 4.0K Jun 

In [ ]:
!ls -lah

total 16K
drwxr-xr-x 1 root root 4.0K Jun 17 16:18 .
drwxr-xr-x 1 root root 4.0K Jun 26 05:15 ..
drwxr-xr-x 1 root root 4.0K Jun 19 16:15 .config
drwxr-xr-x 1 root root 4.0K Jun 17 16:18 sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!cp airbnb-listings.csv "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"

In [ ]:
# aquí creamos nuestra estructura de datos, que va a consistir en la url de la
# imagen y un índice para saber donde insertarla en nuestro array
images_paths = [[i, img_url] for i, img_url in enumerate(full_df['Thumbnail Url'])]
images_paths[:5]

[[0,
  'https://a0.muscache.com/im/pictures/cffe393a-0d84-4fd5-ab4c-a62e067c1b0d.jpg?aki_policy=small'],
 [1,
  'https://a0.muscache.com/im/pictures/ea919e56-aa99-4d5d-a129-1edf0d117d6a.jpg?aki_policy=small'],
 [2,
  'https://a0.muscache.com/im/pictures/57011236/eea5c213_original.jpg?aki_policy=small'],
 [3,
  'https://a0.muscache.com/im/pictures/974f0245-55c2-4e8c-b9bf-14c1c975c798.jpg?aki_policy=small'],
 [4,
  'https://a0.muscache.com/im/pictures/c2dde263-20dd-43af-8c6b-be636c2c0ce1.jpg?aki_policy=small']]

In [ ]:
import imageio as io
import cv2

# esta es la función que se descargará la imagen y devolverá la imagen y el 
# índice indicando la posición donde se incrustará la imagen en nuestro array
def get_image(data_url, target_size=(224, 224)):
    idx, url = data_url
    try:
        img = io.imread(url)
        # hay alguna imagen en blanco y negro y daría error al incluirla en 
        # nuestro array de imagenes que tiene 3 canales, así que convertimos
        # todas las imágenes que tengan menos de 3 dimensiones a color
        if img.ndim < 3:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        img = cv2.resize(img, dsize=target_size)
        return img, idx
    except IOError as err:
        return (None, idx)

In [ ]:
import numpy as np

# en este array iremos incrustando las imágenes conforme las vayamos obteniendo
loaded_images = np.zeros((len(images_paths), 224, 224, 3), dtype=np.uint8)

# y en este array llevaremos un control de cuales se han cargado correctamente
# y cuales no
was_loaded = np.zeros(len(images_paths))

In [ ]:
import concurrent
from tqdm import tqdm

# creamos un pool de procesos que se irán descargando las imágenes
# por defecto, se crearán tantos como CPUs tenga vuestra máquina
with concurrent.futures.ProcessPoolExecutor() as executor:
    # procesamos la lista de urls de imágenes paralelizandola con el pool de procesos
    for (img, idx) in tqdm(executor.map(get_image, images_paths), total=len(images_paths)):
        # metemos la imagen en nuestro array
        if img is not None:
            loaded_images[idx] = img
            was_loaded[idx] = 1
        else:
            was_loaded[idx] = 0

print('Terminado!')
print(f'Total de imágenes recuperadas correctamente: {sum(was_loaded)}/{len(images_paths)}')

100%|██████████| 14001/14001 [08:14<00:00, 28.29it/s]

Terminado!
Total de imágenes recuperadas correctamente: 11271.0/14001


In [ ]:
# guardamos las imágenes (y yo os recomiendo que os lo guardéis en GDrive para evitar tener que repetir esto)
np.save('images.npy', loaded_images)
np.save('was_loaded.npy', was_loaded)

In [ ]:
# almacenamos las imagenes en nuestro drive
!cp images.npy "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"
!cp was_loaded.npy "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"

Esta parte de descarga, montado y copiado solo hace falta ejecutarla la primera vez. Una vez que lo tenemos almacenado en My Drive solo necesitamos cargarlo directamente.

A partir de aquí empieza nuestro ejercicio de regresión.

Como hábito de buena costumbre, para no incurrir en errores involuntarios, en primer lugar se va a dividir el dataset original en train, validation y test.

Se trabaja únicamente con el de train con el objetivo de elegir un modelo. Eso se verifica con el conjunto de validation y finalmente se aplica ese "entrenamiento" al bloque de test.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


#hacemos la divisón en train, val y test
full_df = pd.read_csv('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/airbnb-listings.csv', sep=';', decimal='.')
full_train, test = train_test_split(full_df, test_size=0.2, shuffle=True, random_state=0)
train, val = train_test_split(full_train, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de validación: {val.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./train.csv', sep=';', decimal='.', index=True)
val.to_csv('./val.csv', sep=';', decimal='.', index=True)
test.to_csv('./test.csv', sep=';', decimal='.', index=True)


Dimensiones del dataset de training: (8960, 89)
Dimensiones del dataset de validación: (2240, 89)
Dimensiones del dataset de test: (2801, 89)


In [ ]:
#cargamos las imágenes desde el directorio de My Drive (ya las habíamos descargado previamente)
images  = np.load('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/images.npy')
was_loaded  = np.load('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/was_loaded.npy')

#cargamos los datos ya divididos en train, val y test
df_train = pd.read_csv('./train.csv', sep=';', decimal='.')
df_val = pd.read_csv('./val.csv', sep=';', decimal='.')
df_test = pd.read_csv('./test.csv', sep=';', decimal='.')

#usando el índice de la división anterior obtenemos los conjuntos de test, val y test en las imágenes
train_imgs = images[df_train['Unnamed: 0']]
val_imgs = images[df_val['Unnamed: 0']]
test_imgs = images[df_test['Unnamed: 0']]

train_was_loaded = was_loaded[df_train['Unnamed: 0']]
val_was_loaded = was_loaded[df_val['Unnamed: 0']]
test_was_loaded = was_loaded[df_test['Unnamed: 0']]

print(f'Dimensiones del dataset de training: {train_imgs.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs.shape}')
print(f'Dimensiones del dataset de test: {test_imgs.shape}')

print(f'Dimensiones del dataset de training: {train_was_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_was_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_was_loaded.shape}')

Dimensiones del dataset de training: (8960, 224, 224, 3)
Dimensiones del dataset de validación: (2240, 224, 224, 3)
Dimensiones del dataset de test: (2801, 224, 224, 3)
Dimensiones del dataset de training: (8960,)
Dimensiones del dataset de validación: (2240,)
Dimensiones del dataset de test: (2801,)


In [ ]:
# nos quedamos con los datos e imágenes para los que hemos podido encontrar imágenes
train_imgs_loaded = train_imgs[train_was_loaded == 1]
val_imgs_loaded = val_imgs[val_was_loaded == 1]
test_imgs_loaded = test_imgs[test_was_loaded == 1]

train_with_imgs = df_train[train_was_loaded == 1]
val_with_imgs = df_val[val_was_loaded == 1]
test_with_imgs = df_test[test_was_loaded == 1]

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_imgs_loaded.shape}')

print(f'Dimensiones del dataset de training: {train_with_imgs.shape}')
print(f'Dimensiones del dataset de validación: {val_with_imgs.shape}')
print(f'Dimensiones del dataset de test: {test_with_imgs.shape}')

Dimensiones del dataset de training: (7204, 224, 224, 3)
Dimensiones del dataset de validación: (1790, 224, 224, 3)
Dimensiones del dataset de test: (2277, 224, 224, 3)
Dimensiones del dataset de training: (7204, 90)
Dimensiones del dataset de validación: (1790, 90)
Dimensiones del dataset de test: (2277, 90)


In [ ]:
#PRICE
#imputamos valores vacíos con la media de train
MeanPriceTrain = train_with_imgs['Price'].mean()
train_with_imgs['Price'].fillna(MeanPriceTrain, inplace=True)
val_with_imgs['Price'].fillna(MeanPriceTrain, inplace=True)
test_with_imgs['Price'].fillna(MeanPriceTrain, inplace=True)
#definimos outlier >400€
#indexTrainFiltered = train_with_imgs[train_with_imgs['Price']>400].index
#train_with_imgs.drop(indexTrainFiltered, inplace=True)
#indexValFiltered = val_with_imgs[val_with_imgs['Price']>400].index
#val_with_imgs.drop(indexValFiltered, inplace=True)
#indexTestFiltered = test_with_imgs[test_with_imgs['Price']>400].index
#test_with_imgs.drop(indexTestFiltered, inplace=True)

#train_imgs_loaded.drop(indexTrainFiltered, inplace=True)
#val_imgs_loaded.drop(indexValFiltered, inplace=True)
#test_imgs_loaded.drop(indexTestFiltered, inplace=True)

#transformamos variable Price a gausiana
train_with_imgs['Price'] = train_with_imgs['Price'].apply(lambda x: np.log10(x))
val_with_imgs['Price'] = val_with_imgs['Price'].apply(lambda x: np.log10(x))
test_with_imgs['Price'] = test_with_imgs['Price'].apply(lambda x: np.log10(x))

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_imgs_loaded.shape}')

print(f'Dimensiones del dataset de training: {train_with_imgs.shape}')
print(f'Dimensiones del dataset de validación: {val_with_imgs.shape}')
print(f'Dimensiones del dataset de test: {test_with_imgs.shape}')

Dimensiones del dataset de training: (7204, 224, 224, 3)
Dimensiones del dataset de validación: (1790, 224, 224, 3)
Dimensiones del dataset de test: (2277, 224, 224, 3)
Dimensiones del dataset de training: (7204, 90)
Dimensiones del dataset de validación: (1790, 90)
Dimensiones del dataset de test: (2277, 90)


In [ ]:
#Redimensionamos las imágenes de entrada. Estoy teniendo problemas de RAM y no puedo ejecutarlo
#con 224x224 no puedo escalar /255. Con 112x112 no puedo ejecutar el modelo
#es necesario asumir esta pérdida de información
train_imgs_loaded = np.resize(train_imgs_loaded, (train_imgs_loaded.shape[0],64, 64, train_imgs_loaded.shape[3]))
val_imgs_loaded = np.resize(val_imgs_loaded, (val_imgs_loaded.shape[0],64, 64, val_imgs_loaded.shape[3]))
test_imgs_loaded = np.resize(test_imgs_loaded, (test_imgs_loaded.shape[0],64, 64, test_imgs_loaded.shape[3]))

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de training: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de training: {test_imgs_loaded.shape}')

Dimensiones del dataset de training: (7204, 64, 64, 3)
Dimensiones del dataset de training: (1790, 64, 64, 3)
Dimensiones del dataset de training: (2277, 64, 64, 3)


In [ ]:
#escalamos los datos de entrada. Lo hago en celdas separadas ya que hay algún problema de RAM
#se trata de imágenes así que no hace falta centrar, solo dividimos por el máximo. 
# nos aseguramos de hacerlo como float para no perder la info de los decimales

train_imgs_loaded = train_imgs_loaded.astype('float32') / 255.


In [ ]:
val_imgs_loaded = val_imgs_loaded.astype('float32') / 255.


In [ ]:
test_imgs_loaded = test_imgs_loaded.astype('float32') / 255.

In [ ]:
#extraemos la variable objetivo
Ytrain = train_with_imgs['Price']
Yval = val_with_imgs['Price']
Ytest = test_with_imgs['Price']

Una vez que ya tenemos los datos de las imágenes y la variable objetivo preparados, definimos los modelos con los que vamos a ir trabajando.

La idea general es usar una red neuronal convolucional (CNN) donde las primeras capas actúan como extractor de características y añadimos un clasificador final.

In [ ]:
# imports necesarios
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.layers import BatchNormalization, Activation
from keras.layers import Dropout
from keras.utils import to_categorical

# Inizializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(64, 64, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation="linear"))

# Compilamos el modelo
model.compile(loss='mse',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Entrenamos el modelo
model.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=100,
          validation_data=(val_imgs_loaded, Yval)) 


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.




Train on 7204 samples, validate on 1790 samples
Epoch 1/100
7204/7204 [==============================] - 10s 1ms/step - loss: 6.5384 - root_mean_squared_error: 3.3127 - val_loss: 2.9495 - val_root_mean_squared_error: 2.4749
Epoch 2/100
7204/7204 [==============================] - 3s 386us/step - loss: 1.4493 - root_mean_squared_error: 2.1627 - val_loss: 2.9437 - val_root_mean_squared_error: 1.9560
Epoch 3/100
7204/7204 [==============================] - 3s 386us/step - loss: 0.6527 - root_mean_squared_error: 1.8108 - val_loss: 2.9201 - val_root_mean_squared_error: 1.7000
Epoch 4/100
7204/7204 [==============================] - 3s 386us/step - loss: 0.4295 - root_mean_squared_error: 1.6138 - val_loss: 2.8697 - val_root_mean_squared_error: 1.5446
Epoch 5/100
7204/7204 [==============================] - 3s 388us/step - loss: 0.3295 - root_mean_squared_error: 1.4863 - val_loss: 2.7747 - val_root_mean_squared_error: 1.4379
Epoch 6/100
7204/7204 [==============================] - 3s 388us/

Al tratarse de una regresión volvemos a usar el RMSE para comparar los modelos.

Aquí se observa también que a pesar de aumentar el número de épocas el modelo sigue aprendiendo (por falta de tiempo lo dejamos aquí, pero se podría dejar más tiempo hasta que convergiera del todo).

In [ ]:
predTrain = model.predict(train_imgs_loaded)
predVal = model.predict(val_imgs_loaded)

#deshacemos la transformación logarítmica
predTrain_Eur = pd.DataFrame(predTrain).apply(lambda x: 10**(x))
predVal_Eur = pd.DataFrame(predVal).apply(lambda x: 10**(x))
Ytrain_Eur = pd.DataFrame(Ytrain).apply(lambda x: 10**(x))
Yval_Eur = pd.DataFrame(Yval).apply(lambda x: 10**(x))

#calculamos el MSE y el RMSE para train y test
mseTrainModel = mean_squared_error(Ytrain_Eur,predTrain_Eur)
mseValModel = mean_squared_error(Yval_Eur,predVal_Eur)

print('MSE (train): %0.3g' % mseTrainModel)
print('MSE (val) : %0.3g' % mseValModel)

print('RMSE (train): %0.3g' % np.sqrt(mseTrainModel))
print('RMSE (val) : %0.3g' % np.sqrt(mseValModel))

MSE (train): 7.89e+03
MSE (val) : 6.25e+03
RMSE (train): 88.8
RMSE (val) : 79


In [ ]:
#quito el BatchNormalization

# Inizializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation="linear"))

# Compilamos el modelo
model.compile(loss='mse',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])


# Entrenamos el modelo
model.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=100,
          validation_data=(val_imgs_loaded, Yval)) 

Train on 7204 samples, validate on 1790 samples
Epoch 1/100
7204/7204 [==============================] - 2s 321us/step - loss: 0.3710 - root_mean_squared_error: 0.7828 - val_loss: 1.3954 - val_root_mean_squared_error: 0.6961
Epoch 2/100
7204/7204 [==============================] - 2s 261us/step - loss: 0.2034 - root_mean_squared_error: 0.6882 - val_loss: 1.4364 - val_root_mean_squared_error: 0.6820
Epoch 3/100
7204/7204 [==============================] - 2s 262us/step - loss: 0.1759 - root_mean_squared_error: 0.6761 - val_loss: 1.5538 - val_root_mean_squared_error: 0.6756
Epoch 4/100
7204/7204 [==============================] - 2s 261us/step - loss: 0.1473 - root_mean_squared_error: 0.6720 - val_loss: 1.3771 - val_root_mean_squared_error: 0.6667
Epoch 5/100
7204/7204 [==============================] - 2s 263us/step - loss: 0.1265 - root_mean_squared_error: 0.6600 - val_loss: 1.0654 - val_root_mean_squared_error: 0.6510
Epoch 6/100
7204/7204 [==============================] - 2s 266us/s

In [ ]:
predTrain = model.predict(train_imgs_loaded)
predVal = model.predict(val_imgs_loaded)

#deshacemos la transformación logarítmica
predTrain_Eur = pd.DataFrame(predTrain).apply(lambda x: 10**(x))
predVal_Eur = pd.DataFrame(predVal).apply(lambda x: 10**(x))
Ytrain_Eur = pd.DataFrame(Ytrain).apply(lambda x: 10**(x))
Yval_Eur = pd.DataFrame(Yval).apply(lambda x: 10**(x))

#calculamos el MSE y el RMSE para train y test
mseTrainModel = mean_squared_error(Ytrain_Eur,predTrain_Eur)
mseValModel = mean_squared_error(Yval_Eur,predVal_Eur)

print('MSE (train): %0.3g' % mseTrainModel)
print('MSE (val) : %0.3g' % mseValModel)

print('RMSE (train): %0.3g' % np.sqrt(mseTrainModel))
print('RMSE (val) : %0.3g' % np.sqrt(mseValModel))

MSE (train): 5.01e+03
MSE (val) : 3.71e+03
RMSE (train): 70.8
RMSE (val) : 60.9


In [ ]:
#ahora añadimos MaxNormalization en el clasificador

from keras.constraints import max_norm

# Inizializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_constraint=max_norm(3.)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="linear"))

# Compilamos el modelo
model.compile(loss='mse',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Entrenamos el modelo
model.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=100,
          validation_data=(val_imgs_loaded, Yval)) 

Train on 7204 samples, validate on 1790 samples
Epoch 1/100
7204/7204 [==============================] - 2s 314us/step - loss: 0.3987 - root_mean_squared_error: 0.8235 - val_loss: 1.3591 - val_root_mean_squared_error: 0.7108
Epoch 2/100
7204/7204 [==============================] - 2s 260us/step - loss: 0.2015 - root_mean_squared_error: 0.6965 - val_loss: 1.5191 - val_root_mean_squared_error: 0.6904
Epoch 3/100
7204/7204 [==============================] - 2s 263us/step - loss: 0.1732 - root_mean_squared_error: 0.6853 - val_loss: 1.3467 - val_root_mean_squared_error: 0.6772
Epoch 4/100
7204/7204 [==============================] - 2s 268us/step - loss: 0.1418 - root_mean_squared_error: 0.6684 - val_loss: 1.4694 - val_root_mean_squared_error: 0.6651
Epoch 5/100
7204/7204 [==============================] - 2s 264us/step - loss: 0.1222 - root_mean_squared_error: 0.6596 - val_loss: 1.2784 - val_root_mean_squared_error: 0.6540
Epoch 6/100
7204/7204 [==============================] - 2s 265us/s

In [ ]:
predTrain = model.predict(train_imgs_loaded)
predVal = model.predict(val_imgs_loaded)

#deshacemos la transformación logarítmica
predTrain_Eur = pd.DataFrame(predTrain).apply(lambda x: 10**(x))
predVal_Eur = pd.DataFrame(predVal).apply(lambda x: 10**(x))
Ytrain_Eur = pd.DataFrame(Ytrain).apply(lambda x: 10**(x))
Yval_Eur = pd.DataFrame(Yval).apply(lambda x: 10**(x))

#calculamos el MSE y el RMSE para train y test
mseTrainModel = mean_squared_error(Ytrain_Eur,predTrain_Eur)
mseValModel = mean_squared_error(Yval_Eur,predVal_Eur)

print('MSE (train): %0.3g' % mseTrainModel)
print('MSE (val) : %0.3g' % mseValModel)

print('RMSE (train): %0.3g' % np.sqrt(mseTrainModel))
print('RMSE (val) : %0.3g' % np.sqrt(mseValModel))

MSE (train): 4.92e+03
MSE (val) : 3.72e+03
RMSE (train): 70.1
RMSE (val) : 61


Observamos cómo en los casos anteriores obtenemos mejores resultados en el conjunto de validación que en el de train. No es algo muy habitual, pero se puede dar.

Cogemos el modelo con batchNoramlization y MaxNorm como referencia y usamos Hyper-opt para optimizar nuestros resultados y encontrar las características del modelo que mejor prestaciones nos de.

Vamos a trabajar con las mismas variables que en el notebook de clasificación:

- el número de filtros en las capas convolucionales
- el porcentaje de dropout
- el número de neuronas en la capa densa
- el tamaño del kernel en las capas convolucionales

In [ ]:
# instalamos los paquetes necesarios
!pip install networkx==1.11 # para instala hyperopt correctamente, si no, da errores
!pip install hyperopt

In [ ]:
# imports necesarios
import sys
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.callbacks import EarlyStopping


# definimos nuestro espacio de búsqueda
# vamos a variar:
# - el número de filtros en nuestras capas conv
# - el porcentaje de dropout
# - el número de neuronas en la capa dense
# - el tamaño del kernel en las capas conv
space = {
    'n_filters_conv': hp.choice('n_filters_conv', [32, 64, 128]),
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'neurons_dense': hp.choice('neurons_dense', [256, 512, 1024]), 
    'kernel_size' : hp.choice('kernel_size', [3, 5])
}

def	get_callbacks(pars):
  callbacks	= [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0, mode='auto')]
  return callbacks

def mi_cnn(pars):
  print ('Parameters: ', pars)
  # Inizializamos el modelo
  model = Sequential()

  # Definimos una capa convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(pars['kernel_size']), input_shape=(64, 64, 3)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Definimos una segunda capa convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(pars['kernel_size']), activation='relu'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Definimos una tercera capa convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(pars['kernel_size']), activation='relu'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Añadimos nuestro clasificador
  model.add(Flatten())
  model.add(Dense(pars['neurons_dense'], activation='relu', kernel_constraint=max_norm(3.)))
  model.add(Dropout(pars['dropout']))
  model.add(Dense(1, activation="linear"))

  # Compilamos el modelo
  model.compile(loss='mse',
                optimizer=Adam(lr=0.0001, decay=1e-6),
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  # Entrenamos el modelo
  history = model.fit(train_imgs_loaded, Ytrain,
            batch_size=128,
            shuffle=True,
            epochs=50,
            validation_data=(val_imgs_loaded, Yval),
            verbose = 0,
            callbacks = get_callbacks(pars)) 
  

  best_epoch_loss = np.argmin(history.history['val_loss'])
  best_val_loss = np.min(history.history['val_loss'])
  best_val_rmse = np.max(history.history['val_root_mean_squared_error'])
  
  print('Epoch {} - val rmse: {} - val loss: {}'.format(best_epoch_loss, best_val_rmse, best_val_loss))
  sys.stdout.flush()
  
  return {'loss': best_val_loss, 'best_epoch': best_epoch_loss, 'eval_time': time.time(), 'status': STATUS_OK, 'model': model, 'history': history}


trials = Trials()
best = fmin(mi_cnn, space, algo=tpe.suggest, max_evals=10, trials=trials)
print(best)

Parameters: 
{'dropout': 0.4188545551927214, 'kernel_size': 3, 'n_filters_conv': 64, 'neurons_dense': 512}
Epoch 24 - val rmse: 2.460111141204834 - val loss: 2.1366975331439653
Parameters: 
{'dropout': 0.32696596154925855, 'kernel_size': 3, 'n_filters_conv': 128, 'neurons_dense': 1024}
Epoch 15 - val rmse: 2.6364800930023193 - val loss: 1.7661804998387172
Parameters: 
{'dropout': 0.1503921765380709, 'kernel_size': 3, 'n_filters_conv': 128, 'neurons_dense': 512}
Epoch 13 - val rmse: 1.7771660089492798 - val loss: 0.6980511625385817
Parameters: 
{'dropout': 0.056083081143413094, 'kernel_size': 3, 'n_filters_conv': 64, 'neurons_dense': 256}
Epoch 14 - val rmse: 1.4797413349151611 - val loss: 0.248314151384311
Parameters: 
{'dropout': 0.21145965248878568, 'kernel_size': 5, 'n_filters_conv': 64, 'neurons_dense': 1024}
Epoch 11 - val rmse: 1.4202003479003906 - val loss: 0.950022894909928
Parameters: 
{'dropout': 0.28530178143648355, 'kernel_size': 3, 'n_filters_conv': 32, 'neurons_dense': 10

Viendo los resultados estos son los mejores parámetros que entrenan nuestro modelo:

{'dropout': 0.025691654638946293, 'kernel_size': 1, 'n_filters_conv': 1, 'neurons_dense': 0}

Parameters: 
{'dropout': 0.025691654638946293, 'kernel_size': 5, 'n_filters_conv': 64, 'neurons_dense': 256}
Epoch 11 - val rmse: 1.073818325996399 - val loss: 0.16717064477211938

64 filtros, 256 neuronas y 5x5 el kernel.

In [ ]:
# Inizializamos el modelo definitivo con los parámetros optimizados
modelDef = Sequential()

# Definimos una capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5,5), input_shape=(64, 64, 3)))
modelDef.add(BatchNormalization())
modelDef.add(Activation('relu'))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.02569))

# Definimos una segunda capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
modelDef.add(BatchNormalization())
modelDef.add(Activation('relu'))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.02569))

# Definimos una tercera capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
modelDef.add(BatchNormalization())
modelDef.add(Activation('relu'))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.02569))

# Añadimos nuestro clasificador
modelDef.add(Flatten())
modelDef.add(Dense(256, activation='relu', kernel_constraint=max_norm(3.)))
modelDef.add(Dropout(0.02569))
modelDef.add(Dense(1, activation="linear"))


# Compilamos el modelo
modelDef.compile(loss='mse',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Entrenamos el modelo
modelDef.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=100,
          validation_data=(val_imgs_loaded, Yval)) 

Train on 7204 samples, validate on 1790 samples
Epoch 1/100
7204/7204 [==============================] - 4s 509us/step - loss: 1.0659 - root_mean_squared_error: 1.2719 - val_loss: 2.8887 - val_root_mean_squared_error: 1.1266
Epoch 2/100
7204/7204 [==============================] - 2s 243us/step - loss: 0.4172 - root_mean_squared_error: 1.0747 - val_loss: 2.8282 - val_root_mean_squared_error: 1.0387
Epoch 3/100
7204/7204 [==============================] - 2s 248us/step - loss: 0.3055 - root_mean_squared_error: 1.0120 - val_loss: 2.7598 - val_root_mean_squared_error: 0.9918
Epoch 4/100
7204/7204 [==============================] - 2s 242us/step - loss: 0.2476 - root_mean_squared_error: 0.9733 - val_loss: 2.5835 - val_root_mean_squared_error: 0.9585
Epoch 5/100
7204/7204 [==============================] - 2s 243us/step - loss: 0.2176 - root_mean_squared_error: 0.9438 - val_loss: 2.1331 - val_root_mean_squared_error: 0.9287
Epoch 6/100
7204/7204 [==============================] - 2s 247us/s

In [ ]:
predTrain = modelDef.predict(train_imgs_loaded)
predVal = modelDef.predict(val_imgs_loaded)

#deshacemos la transformación logarítmica
predTrain_Eur = pd.DataFrame(predTrain).apply(lambda x: 10**(x))
predVal_Eur = pd.DataFrame(predVal).apply(lambda x: 10**(x))
Ytrain_Eur = pd.DataFrame(Ytrain).apply(lambda x: 10**(x))
Yval_Eur = pd.DataFrame(Yval).apply(lambda x: 10**(x))

#calculamos el MSE y el RMSE para train y test
mseTrainModel = mean_squared_error(Ytrain_Eur,predTrain_Eur)
mseValModel = mean_squared_error(Yval_Eur,predVal_Eur)

print('MSE (train): %0.3g' % mseTrainModel)
print('MSE (val) : %0.3g' % mseValModel)

print('RMSE (train): %0.3g' % np.sqrt(mseTrainModel))
print('RMSE (val) : %0.3g' % np.sqrt(mseValModel))

MSE (train): 1.47e+03
MSE (val) : 3.82e+03
RMSE (train): 38.3
RMSE (val) : 61.8


Vemos en el resultado que existe un claro overfitting. El RMSE en el conjunto de train es mucho menor que en el de validación, y eso quiere decir que el modelo aprende muy bien los datos de train pero luego generaliza mal.

A pesar de obtener unos parámetros optimizados con Hyper-opt vemos que la complejidad del modelo es muy alta.

En este caso vamos a aumentar un poco el dropout con la intención de aumentar la regulariación, a ver si así disminuimos el overfitting.

Pasamos de 0.02569 a 0.09569

In [ ]:
# Inizializamos el modelo definitivo con los parámetros optimizados
# y con BatchNormalization
modelDef = Sequential()

# Definimos una capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5,5), input_shape=(64, 64, 3)))
modelDef.add(BatchNormalization())
modelDef.add(Activation('relu'))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.09569))

# Definimos una segunda capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
modelDef.add(BatchNormalization())
modelDef.add(Activation('relu'))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.09569))

# Definimos una tercera capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
modelDef.add(BatchNormalization())
modelDef.add(Activation('relu'))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.09569))

# Añadimos nuestro clasificador
modelDef.add(Flatten())
modelDef.add(Dense(256, activation='relu', kernel_constraint=max_norm(3.)))
modelDef.add(Dropout(0.09569))
modelDef.add(Dense(1, activation="linear"))

# Compilamos el modelo
modelDef.compile(loss='mse',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Entrenamos el modelo
modelDef.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=100,
          validation_data=(val_imgs_loaded, Yval)) 



Train on 7204 samples, validate on 1790 samples
Epoch 1/100
7204/7204 [==============================] - 4s 526us/step - loss: 1.4530 - root_mean_squared_error: 1.4736 - val_loss: 2.8732 - val_root_mean_squared_error: 1.2695
Epoch 2/100
7204/7204 [==============================] - 2s 243us/step - loss: 0.6312 - root_mean_squared_error: 1.2021 - val_loss: 2.8347 - val_root_mean_squared_error: 1.1535
Epoch 3/100
7204/7204 [==============================] - 2s 244us/step - loss: 0.4267 - root_mean_squared_error: 1.1163 - val_loss: 2.7282 - val_root_mean_squared_error: 1.0867
Epoch 4/100
7204/7204 [==============================] - 2s 242us/step - loss: 0.3134 - root_mean_squared_error: 1.0600 - val_loss: 2.5866 - val_root_mean_squared_error: 1.0384
Epoch 5/100
7204/7204 [==============================] - 2s 247us/step - loss: 0.2654 - root_mean_squared_error: 1.0184 - val_loss: 2.2168 - val_root_mean_squared_error: 0.9995
Epoch 6/100
7204/7204 [==============================] - 2s 244us/s

In [ ]:
predTrain = modelDef.predict(train_imgs_loaded)
predVal = modelDef.predict(val_imgs_loaded)

#deshacemos la transformación logarítmica
predTrain_Eur = pd.DataFrame(predTrain).apply(lambda x: 10**(x))
predVal_Eur = pd.DataFrame(predVal).apply(lambda x: 10**(x))
Ytrain_Eur = pd.DataFrame(Ytrain).apply(lambda x: 10**(x))
Yval_Eur = pd.DataFrame(Yval).apply(lambda x: 10**(x))

#calculamos el MSE y el RMSE para train y test
mseTrainModel = mean_squared_error(Ytrain_Eur,predTrain_Eur)
mseValModel = mean_squared_error(Yval_Eur,predVal_Eur)

print('MSE (train): %0.3g' % mseTrainModel)
print('MSE (val) : %0.3g' % mseValModel)

print('RMSE (train): %0.3g' % np.sqrt(mseTrainModel))
print('RMSE (val) : %0.3g' % np.sqrt(mseValModel))

MSE (train): 5.21e+03
MSE (val) : 4.53e+03
RMSE (train): 72.2
RMSE (val) : 67.3


Podemos ver que se reduce el overfitting. No obstante los resultados obtenidos no son ninguna maravilla.

Por último evaluamos el modelo con el conjunto de test.

In [ ]:
# Evaluamos el modelo
scores = modelDef.evaluate(test_imgs_loaded, Ytest)

print('Loss: %.3f' % scores[0])
print('RMSE: %.3f' % scores[1])


2277/2277 [==============================] - 0s 197us/step
Loss: 0.249
RMSE: 0.450


In [ ]:
predTrain = modelDef.predict(train_imgs_loaded)
predTest = modelDef.predict(test_imgs_loaded)

#deshacemos la transformación logarítmica
predTrain_Eur = pd.DataFrame(predTrain).apply(lambda x: 10**(x))
predTest_Eur = pd.DataFrame(predTest).apply(lambda x: 10**(x))
Ytrain_Eur = pd.DataFrame(Ytrain).apply(lambda x: 10**(x))
Ytest_Eur = pd.DataFrame(Ytest).apply(lambda x: 10**(x))

#calculamos el MSE y el RMSE para train y test
mseTrainModel = mean_squared_error(Ytrain_Eur,predTrain_Eur)
mseTestModel = mean_squared_error(Ytest_Eur,predTest_Eur)

print('MSE (train): %0.3g' % mseTrainModel)
print('MSE (test) : %0.3g' % mseTestModel)

print('RMSE (train): %0.3g' % np.sqrt(mseTrainModel))
print('RMSE (test) : %0.3g' % np.sqrt(mseTestModel))

MSE (train): 5.21e+03
MSE (test) : 6.03e+03
RMSE (train): 72.2
RMSE (test) : 77.6


Como podemos ver los resultados son francamente malos. Nos está dando un error de más de 70€ y hay que tener en cuenta que el precio medio de los apartamentos (en train) es de 66€.

La principal razón es que por problemas de memoria RAM no podemos ejecutar el modelo introduciendo las imágenes en su tamaño original (224x224) y hemos tenido que reducirlo a 64x64.